In [430]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
loans=pd.read_csv(r"C:\Users\divyam.arora\Desktop\New Folder 3\machine-learning-specialization-master\course-3\datasets\lending-club-data.csv",sep=',',low_memory=False)
loans['safe_loans'] = loans['bad_loans'].apply(lambda x : +1 if x==0 else -1)
loans=loans[loans['home_ownership']!='OTHER']
total_loans=loans[loans['safe_loans']==1]
risky_loans=loans[loans['safe_loans']==-1]
total_loans=total_loans.sample(frac=(len(risky_loans)/len(total_loans)),random_state=1)
total_loans=total_loans.append(risky_loans)
features = ['grade',              # grade of the loan
            'term',               # the term of the loan
            'home_ownership',     # home_ownership status: own, mortgage or rent
            'emp_length',         # number of years of employment
           ]
target = 'safe_loans'
total_loans = total_loans[features + [target]]

total_loans=pd.get_dummies(total_loans, columns=["home_ownership"])
total_loans=pd.get_dummies(total_loans, columns=["term"])
total_loans=pd.get_dummies(total_loans, columns=["emp_length"])
total_loans=pd.get_dummies(total_loans, columns=["grade"])
train_data,test_data=train_test_split(total_loans,test_size=0.2,random_state=2)

In [431]:
def intermediate_node_num_mistakes(labels_in_node):
    # Corner case: If labels_in_node is empty, return 0
    if len(labels_in_node) == 0:
        return 0
    
    # Count the number of 1's (safe loans)
    ## YOUR CODE HERE
    safe_loans=len(labels_in_node[labels_in_node==1])
    
    # Count the number of -1's (risky loans)
    ## YOUR CODE HERE
    risky_loans=len(labels_in_node[labels_in_node==-1])
                
    # Return the number of mistakes that the majority classifier makes.
    ## YOUR CODE HERE
    if safe_loans>risky_loans:
        return risky_loans
    else:
        return safe_loans

In [432]:
example_labels = pd.Series([-1, -1, 1, 1, 1])
if intermediate_node_num_mistakes(example_labels) == 2:
    print('Test passed!')
else:
    print('Test 1 failed... try again!')

# Test case 2
example_labels = pd.Series([-1, -1, 1, 1, 1, 1, 1])
if intermediate_node_num_mistakes(example_labels) == 2:
    print('Test passed!')
else:
    print('Test 2 failed... try again!')
    
# Test case 3
example_labels = pd.Series([-1, -1, -1, -1, -1, 1, 1])
if intermediate_node_num_mistakes(example_labels) == 2:
    print('Test passed!')
else:
    print('Test 3 failed... try again!')

Test passed!
Test passed!
Test passed!


In [433]:
def best_splitting_feature(data, features, target):
    
    target_values = data[target]
    best_feature = None # Keep track of the best feature 
    best_error = 10     # Keep track of the best error so far 
    # Note: Since error is always <= 1, we should intialize it with something larger than 1.

    # Convert to float to make sure error gets computed correctly.
    num_data_points = float(len(data))  
    
    # Loop through each feature to consider splitting on that feature
    for feature in features:
        
        # The left split will have all data points where the feature value is 0
        left_split = data[data[feature] == 0]
        
        # The right split will have all data points where the feature value is 1
        ## YOUR CODE HERE
        right_split = data[data[feature] == 1] 
            
        # Calculate the number of misclassified examples in the left split.
        # Remember that we implemented a function for this! (It was called intermediate_node_num_mistakes)
        # YOUR CODE HERE
        left_mistakes = intermediate_node_num_mistakes(left_split[target])
        #print('no of left mistakes is ',left_mistakes)

        # Calculate the number of misclassified examples in the right split.
        ## YOUR CODE HERE
        right_mistakes = intermediate_node_num_mistakes(right_split[target])
        #print('no of right mistakes is ',left_mistakes)
            
        # Compute the classification error of this split.
        # Error = (# of mistakes (left) + # of mistakes (right)) / (# of data points)
        ## YOUR CODE HERE
        error = ((left_mistakes+right_mistakes)/num_data_points)

        # If this is the best error we have found so far, store the feature as best_feature and the error as best_error
        ## YOUR CODE HERE
        if error < best_error:
            best_feature = feature
            best_error = error
        
    
    return best_feature # Return the best feature we found

In [434]:
feature_Set=total_loans.columns.tolist()

In [435]:
feature_Set.remove('safe_loans')

In [436]:
if best_splitting_feature(total_loans, feature_Set, 'safe_loans') == 'term_ 36 months':
    print('Test passed!')
else:
    print('Test failed... try again!')

Test passed!


In [437]:
def create_leaf(target_values):
    
    # Create a leaf node
    leaf = {'splitting_feature' : None,
            'left' : None,
            'right' : None,
            'is_leaf':True,
            'prediction':None}   ## YOUR CODE HERE
    
    # Count the number of data points that are +1 and -1 in this node.
    num_ones = len(target_values[target_values == +1])
    num_minus_ones = len(target_values[target_values == -1])
    
    # For the leaf node, set the prediction to be the majority class.
    # Store the predicted class (1 or -1) in leaf['prediction']
    if num_ones > num_minus_ones:
        leaf['prediction'] = 1      ## YOUR CODE HERE
    else:
        leaf['prediction'] = -1         ## YOUR CODE HERE
        
    # Return the leaf node        
    return leaf 

In [438]:
def decision_tree_create(data, features, target, current_depth = 0, max_depth = 10):
    remaining_features = features[:] # Make a copy of the features.
    
    target_values = data[target]#output series left in current node
    print("--------------------------------------------------------------------")
    print("Subtree, depth = %s (%s data points)." % (current_depth, len(target_values)))
    

    # Stopping condition 1
    # (Check if there are mistakes at current node.
    # Recall you wrote a function intermediate_node_num_mistakes to compute this.)
    if  intermediate_node_num_mistakes(target_values)== 0:  ## YOUR CODE HERE
        print("Stopping condition 1 reached.")     
        # If not mistakes at current node, make current node a leaf node
        return create_leaf(target_values)
    
    # Stopping condition 2 (check if there are remaining features to consider splitting on)
    if remaining_features ==[]:   ## YOUR CODE HERE
        print("Stopping condition 2 reached.")    
        # If there are no remaining features to consider, make current node a leaf node
        return create_leaf(target_values)    
    
    # Additional stopping condition (limit tree depth)
    if current_depth >=max_depth:  ## YOUR CODE HERE
        print("Reached maximum depth. Stopping for now.")
        # If the max tree depth has been reached, make current node a leaf node
        return create_leaf(target_values)

    # Find the best splitting feature (recall the function best_splitting_feature implemented above)
    ## YOUR CODE HERE
    splitting_feature=best_splitting_feature(data, remaining_features, 'safe_loans')
    print('the new splitting fetaure is:',splitting_feature)

    
    # Split on the best feature that we found. 
    left_split = data[data[splitting_feature] == 0]
    right_split = data[data[splitting_feature] == 1]      ## YOUR CODE HERE
    remaining_features.remove(splitting_feature)
    print("Split on feature %s. (%s, %s)" % (\
                                             splitting_feature, len(left_split), len(right_split)))
    
    # Create a leaf node if the split is "perfect"
    if(len(left_split) == len(data)):
        print("Creating leaf node.")
        return create_leaf(left_split[target])
    if(len(right_split) == len(data)):
        print("Creating leaf node.")
        return create_leaf(left_split[target])

        
    # Repeat (recurse) on left and right subtrees
    left_tree = decision_tree_create(left_split, remaining_features, target, current_depth + 1, max_depth)        
    ## YOUR CODE HERE
    right_tree = decision_tree_create(right_split, remaining_features, target, current_depth + 1, max_depth)

    return {'is_leaf'          : False, 
            'prediction'       : None,
            'splitting_feature': splitting_feature,
            'left'             : left_tree, 
            'right'            : right_tree}

In [439]:
def count_nodes(tree):
    if tree['is_leaf']:
        return 1
    return 1 + count_nodes(tree['left']) + count_nodes(tree['right'])

In [440]:
small_data_decision_tree = decision_tree_create(total_loans,feature_Set,'safe_loans', max_depth = 3)
if count_nodes(small_data_decision_tree) == 15:
    print('Test passed!')
else:
    print('Test failed... try again!')
    print('Number of nodes found                :', count_nodes(small_data_decision_tree))
    print('Number of nodes that should be there : 15') 

--------------------------------------------------------------------
Subtree, depth = 0 (46218 data points).
the new splitting fetaure is: term_ 36 months
Split on feature term_ 36 months. (11582, 34636)
--------------------------------------------------------------------
Subtree, depth = 1 (11582 data points).
the new splitting fetaure is: grade_A
Split on feature grade_A. (11432, 150)
--------------------------------------------------------------------
Subtree, depth = 2 (11432 data points).
the new splitting fetaure is: home_ownership_MORTGAGE
Split on feature home_ownership_MORTGAGE. (5100, 6332)
--------------------------------------------------------------------
Subtree, depth = 3 (5100 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------------------------
Subtree, depth = 3 (6332 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------------------------
Subtree, depth = 2 (150 d

In [441]:
my_decision_tree=decision_tree_create(total_loans,feature_Set,'safe_loans', max_depth = 6)

--------------------------------------------------------------------
Subtree, depth = 0 (46218 data points).
the new splitting fetaure is: term_ 36 months
Split on feature term_ 36 months. (11582, 34636)
--------------------------------------------------------------------
Subtree, depth = 1 (11582 data points).
the new splitting fetaure is: grade_A
Split on feature grade_A. (11432, 150)
--------------------------------------------------------------------
Subtree, depth = 2 (11432 data points).
the new splitting fetaure is: home_ownership_MORTGAGE
Split on feature home_ownership_MORTGAGE. (5100, 6332)
--------------------------------------------------------------------
Subtree, depth = 3 (5100 data points).
the new splitting fetaure is: home_ownership_OWN
Split on feature home_ownership_OWN. (4180, 920)
--------------------------------------------------------------------
Subtree, depth = 4 (4180 data points).
the new splitting fetaure is: home_ownership_RENT
Split on feature home_owners

In [442]:
#x is a dataframe with only one row test case
#tree is a dictionary of root node
#anotate decides if we want to see the path or not.
def classify(tree, x, annotate = False):
    
    # if the node is a leaf node.
    if tree['is_leaf']:
        if annotate: 
            print("At leaf, predicting %s" % tree['prediction'])
        return tree['prediction'] 
    else:
        # split on feature.
        #print(x.columns)
        split_feature_value = x.head(1)[tree['splitting_feature']].values[0]
        if annotate: 
            print("Split on %s = %s" % (tree['splitting_feature'], split_feature_value))
        if split_feature_value == 0:
            return classify(tree['left'], x, annotate)
        else:
            return classify(tree['right'],x,annotate)

In [443]:
test_data.head(1)['safe_loans'].values[0]

-1

In [444]:
print('Predicted class: %s ' % classify(my_decision_tree, test_data.head(1)))

Predicted class: 1 


In [445]:
classify(my_decision_tree, test_data.head(1),True)

Split on term_ 36 months = 1
Split on grade_D = 0
Split on grade_E = 0
Split on grade_F = 0
Split on grade_C = 0
Split on grade_G = 0
At leaf, predicting 1


1

In [446]:
def evaluate_classification_error(tree, data, target):
    # Apply the classify(tree, x) to each row in your data
    prediction =data.apply(lambda x: classify(tree, pd.DataFrame(x).transpose()),axis=1)
    total_examples=len(prediction)
    # Once you've made the predictions, calculate the classification error and return it
    ## YOUR CODE HERE
    mistakes=(prediction!=data[target]).value_counts()[True]
    
    return (mistakes/total_examples)*100
    

In [447]:
error=evaluate_classification_error(my_decision_tree,test_data,'safe_loans')

In [448]:
error

37.094331458243182

In [449]:
def print_stump(tree, name = 'root'):
    split_name = tree['splitting_feature'] # split_name is something like 'term. 36 months'
    if split_name is None:
        print("(leaf, label: %s)" % tree['prediction'])
        return None
    split_feature, split_value = split_name.rsplit('_',1)
    print('                       %s' % name)
    print('         |---------------|----------------|')
    print('         |                                |')
    print('         |                                |')
    print('         |                                |')
    print('  [{0} == 0]               [{0} == 1]    '.format(split_name))
    print('         |                                |')
    print('         |                                |')
    print('         |                                |')
    print('    (%s)                         (%s)' \
        % (('leaf, label: ' + str(tree['left']['prediction']) if tree['left']['is_leaf'] else 'subtree'),
           ('leaf, label: ' + str(tree['right']['prediction']) if tree['right']['is_leaf'] else 'subtree')))

In [450]:
print_stump(my_decision_tree)

                       root
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [term_ 36 months == 0]               [term_ 36 months == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (subtree)


In [451]:
def print_tree(tree, name = 'root'):
    split_name = tree['splitting_feature'] # split_name is something like 'term. 36 months'
    if split_name is None:
        print("(leaf, label: %s)" % tree['prediction'])
        return None
    split_feature, split_value = split_name.rsplit('_',1)
    print('                       %s' % name)
    print('         |---------------|----------------|')
    print('         |                                |')
    print('         |                                |')
    print('         |                                |')
    print('  [{0} == 0]               [{0} == 1]    '.format(split_name))
    print('         |                                |')
    print('         |                                |')
    print('         |                                |')
    print('    (%s)                         (%s)' \
        % (('leaf, label: ' + str(tree['left']['prediction']) if tree['left']['is_leaf'] else 'subtree'),
           ('leaf, label: ' + str(tree['right']['prediction']) if tree['right']['is_leaf'] else 'subtree')))
    if tree['left'] is not None:
        print_tree(tree['left'],'')
    if tree['right'] is not None:
        print_tree(tree['right'],'')

In [452]:
print_stump(my_decision_tree['left'], my_decision_tree['splitting_feature'])

                       term_ 36 months
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [grade_A == 0]               [grade_A == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (subtree)


In [453]:
print_stump(my_decision_tree['left']['left'], my_decision_tree['left']['splitting_feature'])

                       grade_A
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [home_ownership_MORTGAGE == 0]               [home_ownership_MORTGAGE == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (leaf, label: -1)
